# Build a Image Classifier From Scratch
> In this tutorial, we will train an image classifier from scratch instead of using transfer learning. Through the process, we will show a few techniques that can help achieve better performance. 

First, we need some data to work with. For this tutorial, we will use the Imagenette dataset from fastai. This is a subset of the Imangenet dataset. It selects 10 very different classess from the full Imagenet dataset. 

In [3]:
from fastai.vision.all import *
data_path = untar_data(URLs.IMAGENETTE)

ModuleNotFoundError: No module named 'fastprogress'

In [1]:
data_path

NameError: name 'data_path' is not defined